In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.preprocessing.image import img_to_array
import time


In [2]:
model = model_from_json(open("./model.json", "r").read())
model.load_weights('best_model.h5')
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
cap = cv2.VideoCapture(0)  # start webcam

while True:
    start_time = time.time()

    ret, test_image = cap.read()  # reading image from video frame
    if not ret:  # if not getting frames then exit
        break

    gray_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)  # converting the rgb image to grayscale

    faces_detected = face_haar_cascade.detectMultiScale(gray_image)  # detecting faces using haar_cascade
    for (x, y, w, h) in faces_detected:  # coordinates of detected faces
        cv2.rectangle(test_image, (x, y), (x+w, y+h), (255, 0, 0), 2)  # putting a rectangle box around detected face
        roi_gray = gray_image[y:y+w, x:x+h]  # capturing only face from the image
        roi_gray = cv2.resize(roi_gray, (48, 48))  # resizing it to 48x48 (that's what the model wants as input)
        image_pixels = img_to_array(roi_gray)  # converting image to array
        image_pixels = np.expand_dims(image_pixels, axis=0)  # expanding dims as the model needs input in batches

        predictions = model.predict(image_pixels)  # model prediction
        max_index = np.argmax(predictions[0])  # getting emotion index

        emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        emotion_prediction = emotion_detection[max_index]

        cv2.putText(test_image, emotion_prediction, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)  # putting text around face

    cv2.imshow('Emotion Detector', test_image)

    elapsed_time = time.time() - start_time
    # Adjust sleep duration to achieve the desired frame rate
    time.sleep(max(0, 1.0 / 30 - elapsed_time))

    if cv2.waitKey(10) == ord('b'):
        break

cap.release()
cv2.destroyAllWindows()
